In [1]:
from pathlib import Path
import numpy as np
import csv
import re
import cv2

In [2]:
from utils import make_coord

coord = make_coord([1,3])
print(coord)

tensor([[ 0.0000, -0.6667],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.6667]])


In [3]:
# 读取相机参数
def read_calib(calib_file_path):
    with open(calib_file_path, 'r') as calib_file:
        calib = {}
        csv_reader = csv.reader(calib_file, delimiter='=')
        for attr, value in csv_reader:
            calib.setdefault(attr, value)
    return calib

def read_pfm(pfm_file_path):
    with open(pfm_file_path, 'rb') as pfm_file:
        header = pfm_file.readline().decode().rstrip()
        channels = 3 if header == 'PF' else 1
        dim_match = re.match(r'^(\d+)\s(\d+)\s$', pfm_file.readline().decode('utf-8'))
        if dim_match:
            width, height = map(int, dim_match.groups())
        else:
            raise Exception("Malformed PFM header.")

        scale = float(pfm_file.readline().decode().rstrip())
        if scale < 0:
            endian = '<'  # littel endian
            scale = -scale
        else:
            endian = '>'  # big endian

        dispariy = np.fromfile(pfm_file, endian + 'f')

    img = np.reshape(dispariy, newshape=(height, width, channels))
    img = np.flipud(img).astype('uint8')
    # print(img.max(), img.min())
    # cv2.imshow("disparity", img)
    return dispariy, [(height, width, channels), scale]

def create_depth_map(pfm_file_path, calib=None):
    dispariy, [shape, scale] = read_pfm(pfm_file_path)

    if calib is None:
        raise Exception("Loss calibration information.")
    else:
        fx = float(calib['cam0'].split(' ')[0].lstrip('['))
        base_line = float(calib['baseline'])
        doffs = float(calib['doffs'])
        # scale factor is used here
        # d = bf/(d+ doffs)         doffs就是(x_or-x_ol) 两个相机主点在各自图像坐标系x方向上的坐标差
        depth_map = fx * base_line / (dispariy / scale + doffs)
        depth_map = np.reshape(depth_map, newshape=shape)
        depth_map = np.flipud(depth_map).astype('uint8')
        return depth_map

def show(img, win_name='image'):
    if img is None:
        raise Exception("Can't display an empty image.")
    else:
        # cv2.namedWindow(win_name, cv2.WINDOW_NORMAL)
        cv2.imshow(win_name, img)
        cv2.waitKey()
        cv2.destroyWindow(win_name)



In [4]:
pfm_file_dir = Path('demo/Pipes-perfect')
calib_file_path = pfm_file_dir.joinpath('calib.txt')
disp_left = pfm_file_dir.joinpath('disp0.pfm')
disparity, [shape, scale] = read_pfm(disp_left)
disparity = np.reshape(disparity, newshape=shape)
disparity = np.flipud(disparity)
print(disparity)
# calibration information
calib = read_calib(calib_file_path)
# create depth map
depth_map_left = create_depth_map(disp_left, calib)
show(depth_map_left, "depth_map")

[[[       inf]
  [101.098465]
  [101.08243 ]
  ...
  [127.32339 ]
  [127.346344]
  [127.369644]]

 [[       inf]
  [101.08514 ]
  [101.073364]
  ...
  [127.29062 ]
  [127.31209 ]
  [127.33338 ]]

 [[       inf]
  [101.07034 ]
  [101.06523 ]
  ...
  [127.25664 ]
  [127.27663 ]
  [127.29655 ]]

 ...

 [[106.26039 ]
  [106.311005]
  [106.359886]
  ...
  [136.53586 ]
  [136.48761 ]
  [136.49104 ]]

 [[106.23775 ]
  [106.29512 ]
  [106.33731 ]
  ...
  [136.48529 ]
  [136.52405 ]
  [136.50172 ]]

 [[106.192535]
  [106.2522  ]
  [106.3069  ]
  ...
  [136.45667 ]
  [136.51457 ]
  [136.5049  ]]]


C:\Users\Administrator\AppData\Local\Temp\ipykernel_22396\1299993179.py:30: RuntimeWarning: invalid value encountered in cast
  img = np.flipud(img).astype('uint8')


error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1272: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'


In [5]:
import cv2
import numpy as np
from utils import warp_grid
from torchvision import transforms

In [ ]:

# 加载左图像和视差图
left_img = cv2.imread('demo/Pipes-perfect/im0.png')

# disparity_new = disparity.squeeze(-1)
# # 创建x和y映射
# height, width = disparity_new.shape
# x_map = np.float32(np.tile(np.linspace(0, width-1, width), (height, 1)))
# y_map = np.float32(np.tile(np.linspace(0, height-1, height), (width, 1)).T)

# # 考虑到视差，更新x映
# x_map = x_map - disparity_new

# # 使用remap函数进行warp
# warped_img = cv2.remap(left_img, x_map, y_map, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)

disp_tensor = transforms.ToTensor()(disparity.copy()).unsqueeze(0)
left_img_tensor = transforms.ToTensor()(left_img.copy()).unsqueeze(0)

warped_img_tensor = warp(left_img_tensor, disp_tensor)
warped_img = warped_img_tensor.squeeze(0).permute(1, 2, 0).numpy() * 255
# 保存结果
cv2.imwrite('vis/warped.png', warped_img)

/home/yi/anaconda3/envs/torch/lib/python3.9/site-packages/torch/nn/functional.py:4296: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(


True

In [ ]:
# 创建x和y映射
height, width, _ = disparity.shape
x_map = np.float32(np.tile(np.linspace(0, width-1, width), (height, 1)))
y_map = np.float32(np.tile(np.linspace(0, height-1, height), (width, 1)).T)

# 考虑到视差，更新x映射
x_map = x_map - disparity[:,:,0]


# 使用remap函数进行warp
warped_img = cv2.remap(left_img, x_map, y_map, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)

# 保存结果
cv2.imwrite('vis/warped.png', warped_img)

True